In [1]:
from importlib import reload as rel
import numpy as np, pandas as pd, datetime as dt, time
import queue, collections, utils, functions as fn
import OrderPlacement as op, AccountPositions as ap

#### Simulated inputs (from extension)

In [2]:
underlying = 'XLV'
msg = 'n=entryL1 d=long t=m p=0 q=1 u=1 c=10000 b=1h'

In [3]:
try:
    if app.isConnected()==False:
        app.connect("127.0.0.1", 4001, 1)
        print('App reconnected')
    else:
        print('App already connected')
except NameError:
    app = fn.TestApp("127.0.0.1", 4001, 1)
    if app.isConnected()==False:
        print('IB Gateway not connected to IB Server')
    else:
        print('App instantiated & connected')

App instantiated & connected


In [7]:
ibcontract, minTick = fn.make_contract(app, underlying)
order1 = fn.make_order(app, ibcontract, minTick, msg)
# orderid1 = place_new_IB_order(app, ibcontract, order1, orderid=None)

IB error id 50 errorcode 366 string No historical data query found for ticker id:50
IB error id -1 errorcode 504 string Not connected
Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour
Failed to get additional contract details: returning unresolved contract


TypeError: cannot unpack non-iterable Contract object

In [44]:
kws = fn.get_holdings(app, ibcontract)

Exceeded maximum wait for wrapper to confirm finished whilst getting positions


In [ ]:
kws

In [9]:
! python main.py

Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour
Failed to get additional contract details: returning unresolved contract
Traceback (most recent call last):
  File "main.py", line 27, in <module>
    ibcontract, minTick = fn.make_contract(app, underlying)
  File "/Users/evelynestrechinsky/Projects/algoview/src/trading/functions.py", line 53, in make_contract
    resolved_ibcontract, minTick = app.resolve_ib_contract(ibcontract)
TypeError: cannot unpack non-iterable Contract object


#### Historical Market Data

In [7]:
ibcontract, minTick = fn.make_contract(app, underlying)
hist_mkt_data = app.get_IB_historical_data(ibcontract, durationStr='2 D', barSizeSetting='1 hour')

IB error id 50 errorcode 366 string No historical data query found for ticker id:50


#### Live Market Data

In [ ]:
ibcontract, minTick = app.make_contract(underlying)

tickerid = app.start_getting_IB_market_data(ibcontract, whatToShow='BID')
time.sleep(5)
best_bid = app.stop_getting_IB_market_data(tickerid)
best_bid_as_df = best_bid.as_pdDataFrame()
print(best_bid)

In [ ]:
market_data1_as_df = market_data1.as_pdDataFrame()
print(market_data1_as_df)

time.sleep(30)

## stops the stream and returns all the data we've got so far
market_data2 = app.stop_getting_IB_market_data(tickerid)

## glue the data together
market_data2_as_df = market_data2.as_pdDataFrame()
all_market_data_as_df = pd.concat([market_data1_as_df, market_data2_as_df])

## show some quotes
some_quotes = all_market_data_as_df.resample("1S").last()[["bid_size", "bid_price", "ask_price", "ask_size"]]
print(some_quotes.head(10))

## show some trades
some_trades = all_market_data_as_df.resample("10L").last()[["last_trade_price", "last_trade_size"]]
print(some_trades.head(10))

#### Order Placement

In [22]:
ibcontract, order1 = app.parse_message(underlying, msg)
orderid1 = app.place_new_IB_order(ibcontract, order1, orderid=None)
print("Placed market order, orderid is %d" % orderid1)

Getting full contract details from the server... 
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.us
38590758,USO,STK,,0.0,,,ARCA,ARCA,USD,USO,USO,False,,combo:
0,0,0: MKT BUY 1@179769313486231570814527423731704356798070567525844996598917476803157260780028538760589558632766878171540458953514382464234321326889464182768467546703537516986049910576551282076245490090389328944075868508455133942304583236903222948165808559332123348274797826204144723168738177180919299881250404026184124858368.000000 
Getting orderid from IB
Using order id of 6
Placed market order, orderid is 6


#### Account Positions

In [ ]:
app = ap.TestApp("127.0.0.1", 7497, 2)

## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

## get the account name from the position
## normally you would know your account name
accountName = positions_list[0][0] #'DU486001'

## and accounting information
accounting_values = app.get_accounting_values(accountName)
print(accounting_values)

## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

app.disconnect()